# 0. Setting

## 0.1. Installation

In [ ]:
# !pip install python-dotenv
# !pip install python-binance

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from datetime import datetime

In [ ]:
load_dotenv()

api_key = os.environ.get("API_KEY") if os.environ.get("API_KEY") else ""
api_secret = os.environ.get("API_SECRET") if os.environ.get("API_SECRET") else ""

## 0.2. Connecting API

In [ ]:
import requests
import json

url = "https://api.binance.com"
api_call = "/api/v3/ticker/price"
headers = {"content-type": "application/json", "X-MBX-APIKEY": api_key}

response = requests.get(url + api_call, headers=headers)
response = json.loads(response.text)

print(response)

In [ ]:
df = pd.DataFrame.from_records(response)
df

# 1. Binance API

```text
Documentation: https://developers.binance.com/docs/binance-spot-api-docs
```

In [ ]:
class BinanceAPI:
    def __init__(self, api_key=None, api_secret=None):
        self.base_url = "https://api.binance.com"
        self.api_key = api_key
        self.api_secret = api_secret
        
    # ข้อมูลเทียนหรือกราฟแท่งเทียนในอดีตสำหรับคู่การเทรดที่กำหนดตาม Symbol
    def get_klines(self, symbol, interval, limit=1000, start_time=None, end_time=None):
        endpoint = "/api/v3/klines"
        params = {
            'symbol': symbol,
            'interval': interval,
            'limit': limit
        }
        
        if start_time:
            params['startTime'] = start_time
        if end_time:
            params['endTime'] = end_time
            
        response = requests.get(self.base_url + endpoint, params=params)
        return response.json()
    
    def get_n_symbol(self, n) :
        endpoint = "/api/v3/ticker/price"
        headers = {"content-type": "application/json", "X-MBX-APIKEY": self.api_key}
        response = requests.get(self.base_url + endpoint, headers=headers)
        response = json.loads(response.text)
        df = pd.DataFrame.from_records(response)
        return df.loc[:n, "symbol"] 
    
    def get_server_time(self, as_timestamp=False) :
        endpoint = "/api/v3/time"
        response = requests.get(self.base_url + endpoint)
        ts = response.json()["serverTime"]
        if as_timestamp:
            return ts
        time = datetime.fromtimestamp(ts / 1000)
        return time.strftime("%Y-%m-%d %H:%M:%S")
    
    # สถิติการเปลี่ยนแปลงราคา 24 ชั่วโมงสำหรับคู่การเทรดที่กำหนดตาม Symbol
    def get_24hr_ticker(self, symbol):
        endpoint = "/api/v3/ticker/24hr"
        params = {'symbol': symbol}
        response = requests.get(self.base_url + endpoint, params=params)
        return response.json()
    
    # ข้อมูล order book ปัจจุบันสำหรับคู่การเทรดที่กำหนดตาม Symbol
    def get_orderbook(self, symbol, limit=100):
        endpoint = "/api/v3/depth"
        params = {'symbol': symbol, 'limit': limit}
        response = requests.get(self.base_url + endpoint, params=params)
        return response.json()

In [ ]:
api = BinanceAPI(api_key, api_secret)
api

In [ ]:
api.get_server_time()

In [ ]:
api.get_n_symbol(10)

In [ ]:
api.get_klines("ETHBTC", "1m", limit=5)

# 2. Collect Data

In [ ]:
def collect_historical_data(api: BinanceAPI, symbol, interval='1m', days=1, end_time=None):
    if end_time is None:
        end_time = int(api.get_server_time(as_timestamp=True))
    start_time = end_time - (days * 24 * 60 * 60 * 1000)
    
    klines = api.get_klines(
        symbol=symbol,
        interval=interval,
        start_time=start_time,
        end_time=end_time,
    )
    
    # Convert to DataFrame
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Convert data types
    numeric_columns = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    df.drop(["ignore"], axis=1, inplace=True)
    
    return df

In [ ]:
api = BinanceAPI(api_key, api_secret)
symbol = "BNBBTC"

train_df = collect_historical_data(api, symbol, interval="5m", days=4)

test_df = collect_historical_data(api, symbol, interval="5m", days=1, end_time=None)
test_df = test_df[test_df['timestamp'] > train_df['timestamp'].max()]

In [ ]:
train_df

In [ ]:
test_df

<b>Columns</b>

1) <b>open</b>: ราคา *แรกสุด* ที่มีการซื้อขายในช่วงเวลา t
2) <b>high</b>: ราคา *สูงสุด* ที่มีการซื้อขายในช่วงเวลา t
3) <b>low</b>: ราคา *ต่ำสุด* ที่มีการซื้อขายในช่วงเวลา t
4) <b>close</b>: ราคา *สุดท้าย* ที่มีการซื้อขายในช่วงเวลา t

``` 
4 Columns นี้มีการพิจารณาค่าตัวเลขเหมือนกัน เข่น 0.2389 คือ 1 ETH แลกได้ 0.2389 BTC 
```

---

5. <b>volume</b>: จำนวนเหรียญหลักรวมที่มีการซื้อขายในช่วงเวลา t
6. <b>quote_asset_volume</b>: จำนวนเหรียญคู่รวมที่มีการซื้อขายในช่วงเวลา t BTC รวม
7. <b>number_of_trades</b>: จำนวนครั้งที่มีการซื้อขายในช่วงเวลา t
8. <b>taker_buy_base_asset_volume</b>: จำนวนเหรียญหลักรวมที่มีการรีบซื้อในทันทีในช่วงเวลา t
9. <b>taker_buy_quote_asset_volume</b>: จำนวนเหรียญคู่รวมที่มีการรีบซื้อในทันทีในช่วงเวลา t

---

10. <b>timestamp</b>: เวลาเริ่มต้นของการซื้อขาย
11. <b>close_time</b>: เวลาสิ้นสุดของการซื้อขาย

---

In [ ]:
train_df.info()

In [ ]:
train_df["taker_buy_base_asset_volume"] = train_df["taker_buy_base_asset_volume"].astype(float)
train_df["taker_buy_quote_asset_volume"] = train_df["taker_buy_quote_asset_volume"].astype(float)

train_df.info()

In [ ]:
test_df.info()

In [ ]:
test_df["taker_buy_base_asset_volume"] = test_df["taker_buy_base_asset_volume"].astype(float)
test_df["taker_buy_quote_asset_volume"] = test_df["taker_buy_quote_asset_volume"].astype(float)

test_df.info()

# 3. Create Indicators

## 3.1. Moving Average

คำนวณค่าเฉลี่ยแบบเคลื่อนที่ทุกๆ n จุด แล้วดูแนวโน้มค่าเฉลี่ยเหล่านั้น

- Bullish: มีแนวโน้มว่าในอนาคต ราคาสูงขึ้น -> 1 ETH มีแนวโน้มจะได้ BTC มากขึ้น
    - ถ้าเรามี ETH อยู่ เราควรถือไว้ หรือซื้อ ETH เพิ่มเติม
    - ถ้าเรามี BTC อยู่ เราควรขายเพื่อซื้อ ETH
     
- Bearish: มีแนวโน้มว่าในอนาคต ราคาลดลง -> 1 ETH มีแนวโน้มจะได้ BTC น้อยลง
    - ถ้าเรามี ETH อยู่ เราควรขายเพื่อซื้อ BTC
    - ถ้าเรามี BTC อยู่ เราควรถือไว้ หรือซื้อ BTC เพิ่มเติม

In [ ]:
def add_moving_average(df: pd.DataFrame):
    df = df.copy()
    
    price_col = "close"
    
    # SMA
    for period in [20, 50, 200]:
        df[f'SMA_{period}'] = df[price_col].rolling(period, min_periods=1).mean()
    
    # EMA   
    for period in [12, 26]:
        df[f'EMA_{period}'] = df[price_col].ewm(span=period, adjust=False).mean()
    
    # Golden Cross: 50-day SMA crosses above 200-day SMA (Long-term Bullish)
    df['golden_cross'] = ((df['SMA_50'] > df['SMA_200']) & 
                         (df['SMA_50'].shift(1) <= df['SMA_200'].shift(1))).astype(int)
    
    # Death Cross: 50-day SMA crosses below 200-day SMA (Long-term Bearish)
    df['death_cross'] = ((df['SMA_50'] < df['SMA_200']) & 
                        (df['SMA_50'].shift(1) >= df['SMA_200'].shift(1))).astype(int)
    
    # Bullish Cross: 20-day SMA crosses above 50-day SMA (Short-term Bullish)
    df['bullish_cross'] = ((df['SMA_20'] > df['SMA_50']) & 
                          (df['SMA_20'].shift(1) <= df['SMA_50'].shift(1))).astype(int)
    
    # Bearish Cross: 20-day SMA crosses below 50-day SMA (Short-term Bearish)
    df['bearish_cross'] = ((df['SMA_20'] < df['SMA_50']) & 
                          (df['SMA_20'].shift(1) >= df['SMA_50'].shift(1))).astype(int)
    
    # EMA Bullish Cross: 12-day EMA crosses above 26-day EMA (Momentum turning up)
    df['ema_bullish_cross'] = ((df['EMA_12'] > df['EMA_26']) & 
                              (df['EMA_12'].shift(1) <= df['EMA_26'].shift(1))).astype(int)
    
    # EMA Bearish Cross: 12-day EMA crosses below 26-day EMA (Momentum turning down)
    df['ema_bearish_cross'] = ((df['EMA_12'] < df['EMA_26']) & 
                              (df['EMA_12'].shift(1) >= df['EMA_26'].shift(1))).astype(int)
    
    # 0 = Very Bearish, 5 = Very Bullish
    df['trend_strength'] = ((df[price_col] > df['SMA_20']).astype(int) + 
                           (df[price_col] > df['SMA_50']).astype(int) + 
                           (df[price_col] > df['SMA_200']).astype(int) +
                           (df[price_col] > df['EMA_12']).astype(int) +
                           (df[price_col] > df['EMA_26']).astype(int))
    
    # Price Distance from MAs
    # Positive = Above MA 
    # Negative = Below MA
    df['price_sma20_dist'] = ((df[price_col] - df['SMA_20']) / df['SMA_20']).fillna(0)
    df['price_sma50_dist'] = ((df[price_col] - df['SMA_50']) / df['SMA_50']).fillna(0)
    df['price_sma200_dist'] = ((df[price_col] - df['SMA_200']) / df['SMA_200']).fillna(0)
    
    # Price Distance from EMAs
    # Positive = Above MA 
    # Negative = Below MA
    df['price_ema12_dist'] = ((df[price_col] - df['EMA_12']) / df['EMA_12']).fillna(0)
    df['price_ema26_dist'] = ((df[price_col] - df['EMA_26']) / df['EMA_26']).fillna(0)
    
    # Bullish Alignment: SMA_20 > SMA_50 > SMA_200
    df['bullish_alignment'] = ((df['SMA_20'] > df['SMA_50']) & 
                              (df['SMA_50'] > df['SMA_200'])).astype(int)
    
    return df

## 3.2. Relative Strength Index (RSI)

https://www.investopedia.com/terms/r/rsi.asp
- บ่งบอกความแรงของราคา (Momentum) ในช่วงเวลาที่กำหนด ซึ่งมักนิยมใช้ 14 วัน
- มีค่าอยู่ในช่วงระหว่าง 0 ถึง 100

พิจารณา ETHBTC
- RSI สูง (>70) = คนซื้อ ETH ด้วย BTC เยอะมาก 
    - ETH อาจแพงเกินไปที่จะซื้อตอนนี้ 
    - ตอนนี้เราควรขาย ETH และซื้อ BTC
- RSI ต่ำ (<30) = คนขาย ETH เพื่อซื้อ BTC เยอะมาก 
    - ETH อาจถูกเกินไปที่จะขายตอนนี้ 
    - ตอนนี้เราควรซื้อ ETH และขาย BTC
- RSI กลางๆ (~50) = การซื้อขาย ETH/BTC ปกติดี  
    - ตอนนี้ควรรอดูสถานการณ์ก่อน

In [ ]:
def add_rsi(df:pd.DataFrame, period=14):
    
    df = df.copy()
    
    price_col='close'
    
    delta = df[price_col].diff()
    
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Average Gain and Loss with Exponential Moving Average
    avg_gains = gains.ewm(alpha=1/period, adjust=False).mean()
    avg_losses = losses.ewm(alpha=1/period, adjust=False).mean()
    
    # Calculate RSI
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))

    df['rsi'] = rsi
    
    return df

## 3.3. MACD

- ใช้ดูแนวโน้ม (trend) และโมเมนตัม (momentum) ของราคา 
- ดูจากความแตกต่างของค่าเฉลี่ยเคลื่อนที่แบบ EMA สองเส้น (fast: EMA12 ลบกับ slow: EMA26)

พิจารณา ETHBTC
- MACD > 0 แสดงว่าราคากำลังขึ้น หรือก็คือมีแนวโน้มว่าในอนาคต BTC จะมีราคาสูงขึ้น
    - ถ้าเราถือ ETH เราควรถือไว้ รอขายในอนาคต
    - ถ้าเราถือ BTC เราควรขายเพื่อซื้อ ETH
- MACD < 0 แสดงว่าราคากำลังลง หรือก็คือมีแนวโน้มว่าในอนาคต BTC จะมีราคาต่ำขึ้น
    - ถ้าเราถือ ETH เราควรขายเพื่อซื้อ BTC
    - ถ้าเราถือ BTC เราควรถือไว้ รอขายในอนาคต

** ยิ่งค่าห่างจาก 0 ยิ่งมีแนวโน้มที่จะไปทางนั้นๆ สูง

- จังหวะที่ควรซื้อ หรือขาย คือจังหวะที่เส้นของ MACD ตัดกับเส้น MACD_Signal
    - MACD ตัดแล้วขึ้นสูงกว่า MACD_Signal: เป็นช่วงราคากำลังขึ้น
    - MACD ตัดแล้วต่ำกว่า MACD_Signal: เป็นช่วงราคากำลังลง

In [ ]:
def add_macd(df, fast=12, slow=26, signal=9):
    # Calculate EMA fast and slow
    df['ema_fast'] = df['close'].ewm(span=fast, adjust=False).mean()
    df['ema_slow'] = df['close'].ewm(span=slow, adjust=False).mean()
    
    # MACD line
    df['macd'] = df['ema_fast'] - df['ema_slow']
    
    # Signal line
    df['macd_signal'] = df['macd'].ewm(span=signal, adjust=False).mean()
    
    return df

## 3.4. Bollinger Bands

- ประกอบด้วย 3 เส้นหลัก:

1. เส้นกลาง (Middle Band) : SMA ของ close หมายถึงแนวโน้มราคากลาง ๆ ในช่วงเวลาที่กำหนด

2. เส้นบน (Upper Band): SMA + 2SD บ่งบอกขอบเขตราคาที่ "สูงกว่าปกติ" หรือเป็นระดับแนวต้าน

3. เส้นล่าง (Lower Band): SMA - 2SD บ่งบอกขอบเขตราคาที่ "ต่ำกว่าปกติ" หรือเป็นระดับแนวรับ

พิจารณา ETHBTC
- มี 4 Case ที่เกิดขึ้นได้
    1. Close Price ชิด Upper Band: ราคาสูงเกินไปแล้ว อาจมีโอกาสราคาตกลงในเร็ว ๆ นี้
        - ถ้าเราถือ ETH ควรขายเพื่อซื้อ BTC
        - ถ้าเราถือ BTC ควรถือไว้ รอขายในอนาคต
    2. Close Price ชิด Lower Band: ราคาต่ำเกินไปแล้ว อาจมีโอกาสราคาขึ้นในเร็ว ๆ นี้
        - ถ้าเราถือ ETH ควรถือไว้ รอขายในอนาคต
        - ถ้าเราถือ BTC ควรขายเพื่อซื้อ ETH
    3. Upper Band กับ Lower Band เข้ามาชิดกัน: ความผันผวนต่ำ เตรียมเคลื่อนไหว
        - ถ้า Close Price สูงกว่า Upper Band อาจเป็นสัญญาณซื้อ ETH ขาย BTC
        - ถ้า Close Price ต่ำกว่า Lower Band อาจเป็นสัญญาณขาย ETH ซื้อ BTC
    4. Upper Band กับ Lower Band ห่างออกจากกัน: ความผันผวนสูง
        - ถ้า Close Price สูงขึ้น และอยู่ใกล้ Upper Band ควรถือ ETH ต่อเนื่อง หรือซื้อเพิ่ม
        - ถ้า Close Price ลดลง และอยู่ใกล้ Lower Band → ควรขาย ETH ซื้อ BTC

In [ ]:
def add_bollinger(df, period=20, std_dev=2):
    df['bb_middle'] = df['close'].rolling(window=period).mean()
    df['bb_std'] = df['close'].rolling(window=period).std()
    
    df['bb_upper'] = df['bb_middle'] + std_dev * df['bb_std']
    df['bb_lower'] = df['bb_middle'] - std_dev * df['bb_std']

    return df

In [ ]:
def add_all_indicators(df) :
    df = add_moving_average(df)
    df = add_rsi(df)
    df = add_macd(df)
    df = add_bollinger(df)
    return df

# 4. Create Model

## EDA

In [ ]:
ind_train_df = add_all_indicators(train_df)
ind_train_df.set_index("timestamp", inplace=True)

ind_test_df = add_all_indicators(test_df)
ind_test_df.set_index("timestamp", inplace=True)

In [ ]:
ind_train_df

In [ ]:
ind_test_df

In [ ]:
ind_train_df.info()

In [ ]:
ind_train_df.isna().sum()

In [ ]:
print("bb_middle\n", ind_train_df["bb_middle"])
print("bb_std\n", ind_train_df["bb_std"])
print("bb_upper\n", ind_train_df["bb_upper"])
print("bb_lower\n", ind_train_df["bb_lower"])
print("rsi\n", ind_train_df["rsi"])

In [ ]:
ind_train_df.dropna(subset=['bb_lower', 'bb_upper', 'bb_middle', 'bb_std', 'rsi'], inplace=True)

In [ ]:
ind_train_df.isna().sum()

In [ ]:
ind_train_df['price_direction'] = (ind_train_df['close'].shift(-1) > ind_train_df['close']).astype(int)
ind_train_df['price_direction'].value_counts(normalize=True)

- 0 คือ ราคาปิดในอนาคต ***น้อยกว่า*** ราคาปิดปัจจุบัน -> ในตอนนี้ควรขาย
- 1 คือ ราคาปิดในอนาคต ***มากกว่า*** ราคาปิดปัจจุบัน -> ในตอนนี้ควรซื้อ

In [ ]:
ind_test_df.info()

In [ ]:
ind_test_df.isna().sum()

In [ ]:
print("bb_middle\n", ind_test_df["bb_middle"])
print("bb_std\n", ind_test_df["bb_std"])
print("bb_upper\n", ind_test_df["bb_upper"])
print("bb_lower\n", ind_test_df["bb_lower"])
print("rsi\n", ind_test_df["rsi"])

In [ ]:
ind_test_df.dropna(subset=['bb_lower', 'bb_upper', 'bb_middle', 'bb_std', 'rsi'], inplace=True)

In [ ]:
ind_test_df.isna().sum()

In [ ]:
ind_test_df['price_direction'] = (ind_test_df['close'].shift(-1) > ind_test_df['close']).astype(int)
ind_test_df['price_direction'].value_counts(normalize=True)

## Preparing to Train Model

In [ ]:
ind_train_df.columns

In [ ]:
ind_train_df['price_direction'].value_counts(normalize=True)

In [ ]:
random_state = 2025
test_size = 0.25

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

features = [
    'SMA_20', 'SMA_50', 'SMA_200',
    'EMA_12', 'EMA_26',
    'rsi', 
    'macd', 'macd_signal',
    'bb_upper', 'bb_middle', 'bb_lower',
    'trend_strength', 'price_sma20_dist', 'price_sma50_dist',
    'price_sma200_dist',
    'price_ema12_dist', 'price_ema26_dist', 'bullish_alignment'
]

target = 'price_direction'  

In [ ]:
def model_evaluation(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    # print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    # print("Classification Report:\n", classification_report(y_test, y_pred))

    accuracy = accuracy_score(y_test, y_pred)
    # print("Accuracy:", accuracy)

    precision_all = precision_score(y_test, y_pred, average=None)
    precision_class0 = precision_all[0]
    precision_class1 = precision_all[1]

    recall_all = recall_score(y_test, y_pred, average=None)
    recall_class0 = recall_all[0]
    recall_class1 = recall_all[1]

    f1_all = f1_score(y_test, y_pred, average=None)
    f1_class0 = f1_all[0]
    f1_class1 = f1_all[1]

    try:
        if hasattr(model, "predict_proba"):
            y_score = model.predict_proba(X_test)[:, 1]
        else:
            y_score = model.decision_function(X_test)
        roc_auc = roc_auc_score(y_test, y_score)
    except Exception as e:
        roc_auc = None
        
    # print(f"ROC-AUC: {roc_auc:.3f}" if roc_auc is not None else "ROC-AUC: N/A")

    # print(f"Precision (class 0): {precision_class0:.3f}, (class 1): {precision_class1:.3f}")
    # print(f"Recall (class 0): {recall_class0:.3f}, (class 1): {recall_class1:.3f}")
    # print(f"F1-score (class 0): {f1_class0:.3f}, (class 1): {f1_class1:.3f}")
    
    return {
        "accuracy": accuracy,
        "precision_class0": precision_class0,
        "precision_class1": precision_class1,  
        "recall_class0": recall_class0,
        "recall_class1": recall_class1,
        "f1_class0": f1_class0,
        "f1_class1": f1_class1,
        "roc_auc": roc_auc
    }
    

In [ ]:
def show_model_results(lr_evaluation, rf_evaluation, xgb_evaluation, knn_evaluation, svm_evaluation): 
    data = {
        "model": ["LR", "RF", "XGB", "KNN", "SVM"],
        "accuracy": [lr_evaluation["accuracy"], rf_evaluation["accuracy"], 
                    xgb_evaluation["accuracy"], knn_evaluation["accuracy"], 
                    svm_evaluation["accuracy"]],
        "precision_class0": [lr_evaluation["precision_class0"], rf_evaluation["precision_class0"],
                            xgb_evaluation["precision_class0"], knn_evaluation["precision_class0"], 
                            svm_evaluation["precision_class0"]],
        "precision_class1": [lr_evaluation["precision_class1"], rf_evaluation["precision_class1"],
                            xgb_evaluation["precision_class1"], knn_evaluation["precision_class1"],
                            svm_evaluation["precision_class1"]],
        "recall_class0": [lr_evaluation["recall_class0"], rf_evaluation["recall_class0"],
                            xgb_evaluation["recall_class0"], knn_evaluation["recall_class0"],
                            svm_evaluation["recall_class0"]],
        "recall_class1": [lr_evaluation["recall_class1"], rf_evaluation["recall_class1"],
                            xgb_evaluation["recall_class1"], knn_evaluation["recall_class1"],
                            svm_evaluation["recall_class1"]],
        "f1_class0": [lr_evaluation["f1_class0"], rf_evaluation["f1_class0"],
                        xgb_evaluation["f1_class0"], knn_evaluation["f1_class0"],
                        svm_evaluation["f1_class0"]],
        "f1_class1": [lr_evaluation["f1_class1"], rf_evaluation["f1_class1"],
                        xgb_evaluation["f1_class1"], knn_evaluation["f1_class1"],
                        svm_evaluation["f1_class1"]],
        "roc_auc": [lr_evaluation["roc_auc"], rf_evaluation["roc_auc"],
                    xgb_evaluation["roc_auc"], knn_evaluation["roc_auc"],
                    svm_evaluation["roc_auc"]]
    }
    
    results_df = pd.DataFrame(data)
    results_df.set_index("model", inplace=True)
    return results_df

## Model Training V1

### Logistic Regression

In [ ]:
def lr_model_v1(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                        test_size=test_size, shuffle=False, 
                                                        random_state=random_state)
    model = LogisticRegression(random_state=random_state)
    model.fit(X_train, y_train)

    # 3. Scoring
    lr_evaluation = model_evaluation(model, X_train, X_test, y_train, y_test)
    return lr_evaluation


### Random Forest Classification

In [ ]:
def rf_model_v1(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                        test_size=test_size, shuffle=False, 
                                                        random_state=random_state)
    model = RandomForestClassifier(random_state=random_state)
    model.fit(X_train, y_train)

    # 3. Scoring
    rf_evaluation = model_evaluation(model, X_train, X_test, y_train, y_test)
    return rf_evaluation

### XGBoost

In [ ]:
def xgb_model_v1(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                        test_size=test_size, shuffle=False, 
                                                        random_state=random_state)
    model = XGBClassifier(random_state=random_state)
    model.fit(X_train, y_train)

    # 3. Scoring
    xgb_evaluation = model_evaluation(model, X_train, X_test, y_train, y_test)
    return xgb_evaluation

### K-Nearest Neighbors

In [ ]:
def knn_model_v1(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                        test_size=test_size, shuffle=False, 
                                                        random_state=random_state)
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)

    # 3. Scoring
    knn_evaluation = model_evaluation(model, X_train, X_test, y_train, y_test)
    return knn_evaluation

### Support Vector Machine

In [ ]:
def svm_model_v1(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, 
                                                        shuffle=False, random_state=random_state)
    model = SVC(random_state=random_state)
    model.fit(X_train, y_train)

    # 3. Scoring
    svm_evaluation = model_evaluation(model, X_train, X_test, y_train, y_test)
    return svm_evaluation

## Model Summary V1

In [ ]:
lr_evaluation = lr_model_v1(ind_train_df)
rf_evaluation = rf_model_v1(ind_train_df)
xgb_evaluation = xgb_model_v1(ind_train_df)
knn_evaluation = knn_model_v1(ind_train_df)
svm_evaluation = svm_model_v1(ind_train_df)

results_df_v1 = show_model_results(lr_evaluation, rf_evaluation, xgb_evaluation, 
                                   knn_evaluation, svm_evaluation)

results_df_v1

## Model Training V2


- เพิ่ม Hyperparameter Tuning 
- เพิ่ม SMOTE เพราะ Class 0 และ Class 1 มีสัดส่วนต่างกันเกินไป

### Logistic Regression

In [ ]:
def lr_model_v2(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, shuffle=False, random_state=random_state
    )

    # 3. Balance Data
    sm = SMOTE(random_state=random_state)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

    # 4. Hyperparameter Tuning
    param_grid = {
        'C': [0.01, 0.1, 1, 10],
        'solver': ['lbfgs', 'liblinear'],
        'class_weight': ['balanced']
    }
    grid = GridSearchCV(
        LogisticRegression(max_iter=1000, random_state=random_state), 
        param_grid, scoring='accuracy'
    )
    grid.fit(X_train_res, y_train_res)
    # print("Best Params:", grid.best_params_)
    # print("Best CV Score:", grid.best_score_)

    # 5. Train model
    model = LogisticRegression(max_iter=1000, **grid.best_params_, random_state=random_state)
    model.fit(X_train_res, y_train_res)

    # 6. Scoring
    lr_evaluation = model_evaluation(model, X_train_res, X_test, y_train_res, y_test)
    return lr_evaluation

### Random Forest Regression

In [ ]:
def rf_model_v2(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, shuffle=False, random_state=random_state
    )

    # 3. Balance Data
    sm = SMOTE(random_state=random_state)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

    # 4. Hyperparameter Tuning
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [4, 8, 16, None],
        'min_samples_split': [2, 5, 10],
        'class_weight': ['balanced']
    }

    grid = GridSearchCV(RandomForestClassifier(random_state=random_state), param_grid,
                        scoring='accuracy')
    grid.fit(X_train_res, y_train_res)
    # print("Best Params:", grid.best_params_)
    # print("Best CV Score:", grid.best_score_)

    # 5. Train model
    model = RandomForestClassifier(random_state=random_state, **grid.best_params_)
    model.fit(X_train_res, y_train_res)

    # 6. Scoring
    rf_evaluation = model_evaluation(model, X_train_res, X_test, y_train_res, y_test)
    return rf_evaluation

### XGBoost

In [ ]:
def xgb_model_v2(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, shuffle=False, random_state=random_state
    )

    # 3. Balance Data
    sm = SMOTE(random_state=random_state)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

    # 4. Hyperparameter Tuning
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1.0],
    }

    grid = GridSearchCV(
        XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=random_state),
        param_grid, scoring='accuracy'
    )
    grid.fit(X_train_res, y_train_res)
    # print("Best Params:", grid.best_params_)
    # print("Best CV Score:", grid.best_score_)

    # 5. Train model
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss',
                        random_state=random_state, **grid.best_params_)
    model.fit(X_train_res, y_train_res)

    # 6. Scoring
    xgb_evaluation = model_evaluation(model, X_train_res, X_test, y_train_res, y_test)
    return xgb_evaluation

### K-Nearest Neighbors

In [ ]:
def knn_model_v2(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, shuffle=False, random_state=random_state
    )

    # 3. Balance Data (SMOTE เฉพาะ train)
    sm = SMOTE(random_state=random_state)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

    # 4. Hyperparameter Tuning
    param_grid = {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan'],
    }
    grid = GridSearchCV(KNeighborsClassifier(), param_grid, scoring='accuracy')
    grid.fit(X_train_res, y_train_res)
    # print("Best Params:", grid.best_params_)
    # print("Best CV Score:", grid.best_score_)

    # 5. Train model
    model = KNeighborsClassifier(**grid.best_params_)
    model.fit(X_train_res, y_train_res)

    # 6. Scoring
    knn_evaluation = model_evaluation(model, X_train_res, X_test, y_train_res, y_test)
    return knn_evaluation

### Support Vector Machine

In [ ]:
def svm_model_v2(df):
    X = df[features]
    y = df[target]
    
    # 1. Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, shuffle=False, random_state=random_state
    )

    # 3. Balance Data
    sm = SMOTE(random_state=random_state)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

    # 4. Hyperparameter Tuning
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto'],
        'class_weight': ['balanced']
    }
    grid = GridSearchCV(SVC(), param_grid, scoring='accuracy')
    grid.fit(X_train_res, y_train_res)
    # print("Best Params:", grid.best_params_)
    # print("Best CV Score:", grid.best_score_)

    # 5. Train model
    model = SVC(**grid.best_params_, random_state=random_state)
    model.fit(X_train_res, y_train_res)

    # 6. Scoring
    svm_evaluation = model_evaluation(model, X_train_res, X_test, y_train_res, y_test)
    return svm_evaluation

## Model Result V2

In [ ]:
lr_evaluation_v2 = lr_model_v2(ind_train_df)
rf_evaluation_v2 = rf_model_v2(ind_train_df)
xgb_evaluation_v2 = xgb_model_v2(ind_train_df)
knn_evaluation_v2 = knn_model_v2(ind_train_df)
svm_evaluation_v2 = svm_model_v2(ind_train_df)

results_df_v2 = show_model_results(lr_evaluation_v2, rf_evaluation_v2, xgb_evaluation_v2, 
                                   knn_evaluation_v2, svm_evaluation_v2)

results_df_v2

## Comparison

In [ ]:
metrics = ['accuracy', 'precision_class0', 'precision_class1', 'recall_class0', 
           'recall_class1', 'f1_class0', 'f1_class1', 'roc_auc']

models = results_df_v1.index.tolist()
x = np.arange(len(models))
width = 0.3

fig, axes = plt.subplots(len(metrics), 1, figsize=(12, 4 * len(metrics)), sharey=True)

for i, metric in enumerate(metrics):
    axes[i].bar(x - width/2, results_df_v1[metric], width, label='V1')
    axes[i].bar(x + width/2, results_df_v2[metric], width, label='V2')
    axes[i].set_title(metric)
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(models, rotation=20)
    axes[i].legend()
    
plt.tight_layout()
plt.show()

# 5. Picking Model

ทดสอบกับ 20 เหรียญ แล้วหา Model ที่ดีที่สุด

In [ ]:
symbols = list(api.get_n_symbol(20))

In [ ]:
def create_train_test_ind_df(symbol):
    api = BinanceAPI(api_key, api_secret)
    train_df = collect_historical_data(api, symbol, interval="5m", days=4)
    test_df = collect_historical_data(api, symbol, interval="5m", days=1, end_time=None)
    test_df = test_df[test_df['timestamp'] > train_df['timestamp'].max()]
    
    ind_train_df = add_all_indicators(train_df)
    ind_train_df.set_index("timestamp", inplace=True)
    ind_train_df.dropna(subset=['bb_lower', 'bb_upper', 'bb_middle', 'bb_std', 'rsi'], inplace=True)
    ind_train_df['price_direction'] = (ind_train_df['close'].shift(-1) > ind_train_df['close']).astype(int)
    
    ind_test_df = add_all_indicators(test_df)
    ind_test_df.set_index("timestamp", inplace=True)
    ind_test_df.dropna(subset=['bb_lower', 'bb_upper', 'bb_middle', 'bb_std', 'rsi'], inplace=True)
    ind_test_df['price_direction'] = (ind_test_df['close'].shift(-1) > ind_test_df['close']).astype(int)
    
    return ind_train_df, ind_test_df

In [ ]:
train_df1, test_df1 = create_train_test_ind_df(symbols[0])
train_df2, test_df2 = create_train_test_ind_df(symbols[10])

In [ ]:
train_df1.head()

In [ ]:
train_df2.head()